# nltk - 텍스트 분석 개요 

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 2장. Accessing Text Corpora and Lexical Resources(https://www.nltk.org/book/ch02.html) 내용 중 일부 인용. 
    - nltk book 3장. Processing Raw Text(https://www.nltk.org/book/ch03.html) 내용 중 일부 인용.     

### lexicon이란 무엇일까? 텍스트 분석이란 lexicon을 찾아가는 과정입니다.
- lexicon(또는 lexical resource): 품사(POS) 등 연관 정보를 갖고 있는 단어 또는 구(phrase) 집합을 말합니다.
- lexicon은 텍스트 분석을 진행해 갈수록 상세한 정보를 갖게 됩니다.
    - 예: 텍스트에 사용된 단어와 빈도 수.
    - 예: concordance 메소드를 사용해 단어가 텍스트에서 어떻게 사용되었는지를 알 수 있습니다.
- lexicon = headword (또는 lemma, 원형) + 품사 + 정의(뜻). 
    - 예: saw(보았다) = saw + verb + 'past tense of see'. 
    - saw(톱) = saw + noun + 'cutting instrument'

In [ ]:
sentence = """
In this book authored by Sohom Ghosh and Dwight Gunning, we shall learnning how to pracess Natueral Language and 
extract insights from it. The first four chapter will introduce you to the basics of NLP. Later chapters will describe 
how to deal with complex NLP prajects. If you want to get early access of it, you should book your order now.
"""

## 문장 인식
- 원시 텍스트는 몇 개의 문장으로 이루어져 있을까요? 
- nltk의 **sent_tokenize()** 와 **LineTokenizer()** 가 문장을 인식하는 클래스입니다.

In [ ]:
from nltk.tokenize import sent_tokenize

sent_list  = sent_tokenize(sentence)
print(len(sent_list))
print(sent_list[-1][:50], '...')

- 이번엔 **LineTokenizer()** 를 사용해 볼까요? 결과가 조금 이상하죠.
- **LineTokenizer()** 는 말 그대로 줄(line)을 셉니다. 
- 원시 텍스트에서 문장을 추출하려면 **sent_tokenize()** 를 사용해야 합니다.

In [ ]:
from nltk.tokenize import LineTokenizer

lTokenizer = LineTokenizer()
sent_list = lTokenizer.tokenize(sentence)
print(len(sent_list))
print(sent_list[-1][:50], '...')

## 단어(token) 인식

- 가장 간단한 토큰 찾는 방법 : split() 메소드를 사용하여 whitespace를 기준으로 쪼개면 됩니다. 
- 그러나 실행 결과는 불만족스럽죠. 쓸모없는 newline 기호('\n')가 포함되었기 때문입니다.

In [ ]:
import nltk, re

In [ ]:
print(re.split(r' ', sentence))

- nltk의 **word_tokenize()** 는 문장에서 token 을 찾을 때 사용하는 메소드입니다.
    - 구둣점도 하나의 token으로 인식합니다.   
- 정규표현의 '\\w+'를 사용하여 token을 찾으면 구둣점은 token에서 제외됩니다.

In [ ]:
from nltk import word_tokenize

print(sent_list[-1])
words = word_tokenize(sent_list[-1])
print(len(words), words[:12])

정규표현의 경우 2개의 구둣점(comma, period)이 token 리스트에서 제외됩니다.

In [ ]:
words = re.findall(r'\w+', sent_list[-1])
print(len(words), words[:12])

In [ ]:
words = nltk.regexp_tokenize(sent_list[-1], r'\w+')
print(len(words), words[:12])

- **nltk.regexp_tokenize()** 는 **re.findall()** 과 기능은 거의 비슷합니다. 
    - 다만, **regexp_tokenize()** 가 더 효율적이며, 특히 괄호 처리에 대한 고민을 덜어줍니다.

- word_tokenize 보다는 정규표현으로 token의 패턴을 찾는 경우가 더 자주 사용한다. 아래 3가지 사용 예를 살펴 보겠습니다.
- 아래 예에서 원시 텍스트에서 **a ___ man** 이 나타난 구(phrase)를 찾습니다.
    - 각괄호(angle bracket)는 토큰 경계를 표시한다. 각괄호 사이의 whitespace는 무시합니다.
    - 단일 토큰 *a* 와 *man* 사이에 \.\* 를 각괄호로 묶었기 때문에, 이 부분에 해당하는 문자열만 출력됩니다.
        - 예: a monied man -> monied

In [ ]:
from nltk.corpus import gutenberg, nps_chat
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))
moby.findall(r"<a> (<.*>) <man>")

아래 예는 3단어로 이루어진 구(phrase)이며, 마지막 단어가 'bro'인 구를 찾습니다.

In [ ]:
chat = nltk.Text(nps_chat.words())
chat.findall(r"<.*> <.*> <bro>")

알파벳 'l'(엘)로 시작하는 3개 이상 단어로 이루어진 구를 찾습니다.

In [ ]:
chat.findall(r"<l.*>{3,}")

nltk의 **TweetTokenizer** 클래스는 SNS 등에 사용되는 token을 인식합니다.

In [ ]:
from nltk.tokenize import TweetTokenizer

tTokenizer = TweetTokenizer()
words = tTokenizer.tokenize("This is a cooool #dummysmiley: :-) :-P <3")
print(len(words), words)

## POS tagging
- POS는 품사(part of sentence)를 말합니다.
- 우리가 찾은 token이 문장에서 어떤 역할을 하는 품사인지 파악하는 과정입니다.
- token의 품사(POS)를 찾으면 token과 POS는 tuple 형태로 묶이게 된다. 이를 POS tagging이라고 부릅니다.
- nltk의 **pos_tag()** 메소드를 사용하여 POS tagging을 실행할 수 있습니다.

In [ ]:
s1 = nltk.word_tokenize("And now for something completely different")
nltk.pos_tag(s1)

- POS tagging 결과는 tuple의 리스트입니다. tuple은 (단어, 품사)로 이루어져 있습니다.
    - And-**CC** (관계 접속사, coordinating conjunction), now, completely-**RB** (부사, adverb)
    - for-**IN** (전치사, preposition), something-**NN** (명사, noun), different-**JJ** (형용사, adjective)

upenn(미국 펜실배니아주립대학, University of Pennsylvania)에서 POS tag에 대해 정리해놓았습니다.

In [ ]:
nltk.help.upenn_tagset('RB')

- POS tagging은 단어를 품사로 분류(categorize)하는 작업입니다.
    - 이렇게 분류하는 게 얼마나 효과적일까? 아래 예제를 통해 이런 의구심을 조금이나마 줄여보도록 하죠.
    - woman(명사), bought(동사), over(전치사), 및 the(관사)에 대해 유사 단어를 찾아봅시다.
    - text.similar() 메소드를 사용하면 됩니다. 이 메소드는 w와 같은 문맥(w1-w-w2)을 갖는 유사 단어를 찾아줍니다.
        - 어디서 많이 본 듯 하죠. nltk의 similar() 메소드와 같은 기능입니다.

- 아래 예에서 brown은 미국 Brown University에서 만든 말뭉치입니다.
    - Brown 말뭉치 매뉴얼 : http://icame.uib.no/brown/bcm.html
    - 미국 브라운 대학교(Brown University)에서 1961년에 최초로 만든 전자 말뭉치(million-word electronic corpus)입니다.
    - 500개 이상의 source로부터 수집한 텍스트로 구성했으며, 뉴스, 종교, 취미, 미스테리 등 다양한 장르(genre)로 구분하고 있습니다.
    - 보충 강의 자료(**nltk-말뭉치**)를 참고하면에 nltk에서 제공하는 말뭉치에 대해 자세히 알 수 있습니다.

In [ ]:
text = nltk.Text(word.lower() for word in nltk.corpus.brown.words())

text.similar('woman')

In [ ]:
text.similar('bought')
#text.similar('over')
#text.similar('the')

## 불용어(stop words) 제거 
- the와 같은 불용어(stopwords)는 lexicon에서 거의 의미가 없기 때문에, 텍스트의 특징을 분석하거나 다른 텍스트와 구분할 때 오히려 방해만 됩니다.
- 대부분의 자연어 전처리과정에서 불용어는 가장 먼저 제거합니다.

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
print(len(stopwords), stopwords[:10])

불용어를 원시 텍스트에서 제거하는 코드는 1줄입니다.

In [ ]:
#sentence는 첫 번째 셀에 있는 원시 텍스트입니다.
words = word_tokenize(sent_list[-1])

new_words = [wd for wd in words if wd not in stopwords]
print(len(new_words), new_words[:10]) #17개에서 9개로 줄었네요!

- **Reuters 말뭉치**
    - 130만개 단어로 이루어진 10,788개 뉴스가 있습니다.
    - 뉴스는 90개 토픽(topic)으로 나누어지며, 각 토픽은 training set과 test set으로 나뉘어 있습니다.
        - training set과 test set으로 나눈 목적은 인공지능 알고리즘을 사용해 뉴스를 자동으로 구분하기 위해서입니다.

In [ ]:
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    print('%2.2f%%' % round(len(content)/len(text)*100, 4))

reuters 말뭉치에서 불용어가 차지하는 비율이 26.27%(=100-73.52)임을 알 수 있습니다.

In [ ]:
content_fraction(nltk.corpus.reuters.words())

- 영단어 사전과 Jane Austen의 소설 Sense and Sensibility(초판 발간, 1811년)에서 사용한 단어를 비교해 보겠습니다.
    - 즉 Austen의 소설에서 영단어 사전에 없는 단어만을 찾습니다.
    - 이 단어들은 현재 사용되지 않거나 철자가 틀린 단어들일 가능성이 높습니다.

2개 리스트를 비교하기 보다는 2개 set(집합)을 비교하는 게 편합니다. 차집합(difference) 연산이 가능하기 때문이죠!

In [ ]:
def unusual_words(text):
    text_vocab = set(w.lower() for w in text if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    unusual = text_vocab - english_vocab #set difference
    return sorted(unusual)

- **Gutenberg 말뭉치**
    - NLTK는 Gutenberg 프로젝트(http://www.gutenberg.org/)의 25,000여개 전자책(무료) 중 18개의 텍스트를 말뭉치로 제공하고 있습니다.

In [ ]:
unused_words = unusual_words(nltk.corpus.gutenberg.words('austen-sense.txt'))
print(len(unused_words))
print(unused_words[:8]) #복수형, 과거형 등도 포함 

- 무려 2,000여개 단어가 다르네요... 그러나 정확히 말해 이 숫자는 과장된 것입니다.
    - 단어의 원형(stem)을 구하지 않고 비교했구요. 또, 복수형, 과거형도 다른 단어로 판단했습니다.
    - *abou*와 같이, 지금은 쓰지 않는 단어들을 찾으려면 조금 더 섬세한 처리가 필요하겠죠.

In [ ]:
unused_words = unusual_words(nltk.corpus.nps_chat.words())
print(len(unused_words))
print(unused_words[:8]) 

## 철자 수정 
- Anaconda prompt에서 **pip install autocorrect** 명령을 실행하여 autocorrect 모듈을 먼저 설치해야 합니다.

sleapy, tehre, giong 등 3개 단어를 정확히 수정했네요.

In [ ]:
from autocorrect import Speller

spell = Speller()
spell("I'm not sleapy and tehre is no place I'm giong to.")

위 과정을 알기 쉽게 코드로 구현해보겠습니다.

In [ ]:
import string

wds = word_tokenize("I'm not sleapy and tehre is no place I'm giong to.")
corrected_sent = ""
corrected_wds = []

for wd in wds:
    tmp = wd
    if wd not in string.punctuation:
        wd_c = spell(wd)
        if wd_c != wd:
            print(wd + " has been corrected to: "+ wd_c)
            tmp = wd_c
        corrected_wds.append(tmp)   
    corrected_sent = corrected_sent + " " + tmp
    
print(corrected_sent)    
print(corrected_wds) 

## 텍스트 정규화 - 서로 다른 표현을 일치시킴
- 정규화 예: 
    - non-standard words, 즉, 숫자, 날짜, 시간, 약어(acronym, ASCII, NASA 등) 등 표현을 일치시킴..
    - 텍스트를 모두 소문자로 변환(The와 the를 구분하지 않음)
- 어간 추출: 어간(stemming)을 찾아 사전에 나오는 단어들로 변환(lemmatization)
    - **stemmer**와 **lemmatizer**(사전 원형 변환) 2종류가 있지만 stemmer를 주로 사용

In [ ]:
sample = "I visited US from UK on 01-08-20"
normalized_sample = sample.replace("US", "United States").replace("UK","United Kingdom").replace("-20", "-2020")
normalized_sample

### 어간 추출(Stemmer)

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
 is no basis for a system of government.  Supreme executive power derives from
 a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = word_tokenize(raw)

nltk에는 PorterStemmer와 LancasterStemmer 등 2종류의 stemmer가 있으나, PorterStemmer가 성능이 더 나은 걸로 알려져 있음.

In [ ]:
from nltk import PorterStemmer, LancasterStemmer

Porter stemmer는 lying의 어간을 lie로 찾았지만, Lancaster stemmer는 lying으로 처리

In [ ]:
porter = PorterStemmer()
stems = [porter.stem(t) for t in tokens]
print(len(stems), stems[:10])

In [ ]:
lancaster = LancasterStemmer()
lstems = [lancaster.stem(t) for t in tokens]
print(len(lstems), lstems[:10])

In [ ]:
stems = [porter.stem(t) for t in corrected_wds]
print(stems)

### 사전 원형 변환(Lemmatization): WordNet lemmatizer
- 단어가 사전에 있으면 접미사(affix)를 없앱니다. 
- lemmatizer가 stemmer보다 사전 전체를 비교하므로 속도가 느립니다.
- lying은 처리하지 못하지만 women은 woman으로 변환합니다.

In [ ]:
from nltk import WordNetLemmatizer

In [ ]:
wordlemma = WordNetLemmatizer()
print(wordlemma.lemmatize('cars'))
print(wordlemma.lemmatize('walking',pos='v'))
print(wordlemma.lemmatize('meeting',pos='n'))
print(wordlemma.lemmatize('meeting',pos='v'))
print(wordlemma.lemmatize('better',pos='a'))

In [ ]:
print([wordlemma.lemmatize(t) for t in tokens])

## 개체 이름 인식(NER, Named Entity Recognition)
- Seoul Botanical Garden은 3단어이지만, 이 3단어는 하나의 개체(entity, 고유명사)입니다.
- 이렇게 개체 이름을 인식하는 것을 **NER**이라고 부릅니다.
    - 여러 개 품사를 하나로 묶어야 개체 이름을 인식하기 때문에 이 과정을 **chunking** 이라고 부릅니다.
    - nltk에서는 **ne_chunk** 가 이러한 기능을 실행합니다.

In [ ]:
sentence = "Seoul Botanical Garden is a well known place in Seoul, Korea."

*Seoul Botanical Garden*을 하나로 인식하지 못하고, *Botanical Garden*만 chunking 대상에 포함되었습니다.

In [ ]:
sentence = "Seoul Botanical Garden is a well known place in Seoul, Korea."

words = nltk.word_tokenize(sentence)
tags = nltk.pos_tag(words)
chunks = nltk.ne_chunk(tags)
print(chunks)

## 단어 중의성 (ambiguity)
- 문장이 중의성(double meaning)을 포함하면 그 뜻이 모호하기 때문에 분석하기 어렵겠죠.
- 같은 단어이지만, 전혀 다른 의미로 사용되는 경우가 자주 있죠.

### WordNet: 의미 지향 사전으로 동의어 사전(thesaurus)과 유사.
- 미국 프린스턴 대학교(Princeton University)에서 1980년대 중반부터 구축한 동의어 사전 - 아쉽게 실패로 끝났습니다!
- NLTK에서는 155,287개 단어와 117,659 동의어(synonym) 집합으로 구성되어 있습니다. 
    - "Benz is credited with the invention of the motorcar." 
        - 위 문장에서 motorcar 대신 automobile로 바꾸더라도 문장의 의미는 바뀌지 않습니다.
        - 따라서 motorcar와 automobile는 동의어(synonym)로 판단할 수 있습니다.

In [ ]:
from nltk.corpus import wordnet as wn
wn.synsets('motorcar')

개체 car.n.01을 **synset** (synonym set, 동의어 단어(lemma) 집합)이라 부릅니다.

In [ ]:
wn.synset('car.n.01').lemma_names()

In [ ]:
print(wn.synset('car.n.01').definition())
print(wn.synset('car.n.01').examples())
print()
print(wn.synset('car.n.01').lemmas())

In [ ]:
print(wn.synsets('car'))
for synset in wn.synsets('car'):
    print(synset.lemma_names())

- nltk에서는 Lesk 알고리즘을 구현해 라이브러리로 제공하고 있습니다. 
    - 1986년 Michael E. Lesk가 제안한 단어 중의성의 모호함을 해결(WSD, word sense disambiguation) 하기 위한 알고리즘입니다. 
    - 아래 예는 단어 bank가 2가지 의미, 즉 은행과 (도로) 갓길로 사용되고 있습니다. 

In [ ]:
from nltk.wsd import lesk

sentence1 = "Keep your savings in the bank"
print(lesk(word_tokenize(sentence1), 'bank'))

sentence2 = "It's so risky to drive over the banks of the road"
print(lesk(word_tokenize(sentence2), 'bank'))

In [ ]:
wn.synsets('bank')[:8]

Lesk 알고리즘에서 제시한 bank.n.07의 단어 정의를 보면 분명하죠!

In [ ]:
print(wn.synset('bank.n.07').definition())